# Langfuse 프롬프트 관리 

---

## 환경 설정 및 준비

### (1) Env 환경변수

```markdown
LANGFUSE_SECRET_KEY=sk-
LANGFUSE_PUBLIC_KEY=pk-
# 🇪🇺 EU region
LANGFUSE_HOST=https://cloud.langfuse.com
```


In [5]:
from dotenv import load_dotenv #사내회사망에서 랭퓨즈를 사용할 수 있다.
load_dotenv()

True

### (2) 기본 라이브러리

In [6]:
import os
from glob import glob
from pprint import pprint
import json
import warnings
warnings.filterwarnings("ignore")

### (3) Langfuse 콜백 핸들러 설정

In [28]:
from langfuse.langchain import CallbackHandler 

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

### (4) Langfuse 클라이언트 설정

In [11]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

---

## 프롬프트 관리 개요

Langfuse는 **프롬프트 CMS(Content Management System)** 기능을 제공

- **버전 관리**: 프롬프트의 모든 변경사항을 추적하고 롤백 가능
- **협업**: 팀원들과 함께 프롬프트를 편집하고 관리
- **배포 관리**: 라벨을 통해 코드 변경 없이 환경별 배포
- **성능 모니터링**: 프롬프트 버전별 성능 메트릭 비교
- **실시간 테스트**: 플레이그라운드에서 즉시 테스트 가능

---

## 1. 프롬프트 생성

### 1.1 텍스트 프롬프트 생성

In [14]:
from langfuse import Langfuse # 1. 라이브러리에서 Langfuse 클래스를 가져옴
import os



# 3. 'langfuse'라는 이름의 객체를 실제로 생성 (이게 빠져서 에러가 난 겁니다!)
langfuse = Langfuse() 

# 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic",  # 프롬프트 이름
    type="text",          
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?", 
    labels=["production"],           # 프로덕션 레이블
    tags=["movie", "qa", "text"],    # 태그
    config={
        "model": "gpt-4.1-nano",
        "temperature": 0.7,
        "max_tokens": 500}



)

### 1.2 챗 프롬프트 생성

In [15]:
# 챗 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic-chat",  # 프롬프트 이름
    type="chat",          
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}를 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "chat"],    # 태그
    config={
        "model": "gpt-4.1-nano",
        "temperature": 0.7,
        "max_tokens": 500
    }
)

### 1.3 메시지 플레이스홀더가 있는 챗 프롬프트

In [22]:
# 메시지 플레이스홀더를 포함한 챗 프롬프트
langfuse.create_prompt(
    name="movie-critic-with-history",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다2."
        },
        {
            "type": "placeholder",
            "name": "chat_history"  # 대화 히스토리 삽입 지점
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대해 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],
    tags=["movie", "qa", "chat", "history"]
)

In [75]:
# 메시지 플레이스홀더를 포함한 챗 프롬프트
langfuse.create_prompt(
    name="keyword-extractor-few-shot",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다2."
        },
        {
            "type": "placeholder",
            "name": "chat_history"  # 대화 히스토리 삽입 지점
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대해 어떻게 생각하시나요?"
        }
    ],
    labels=["production"],
    tags=["qa", "chat", "history"]
)

### **[실습 1]**
텍스트 기반 프롬프트와 chat 기반 프롬프트를 각각 구현하고, Langfuse UI에서 확인하세요.

In [17]:
# 텍스트 프롬프트 생성
# 여기에 코드를 작성하세요


# 텍스트 프롬프트 생성


langfuse.create_prompt(
    name="text_prompt",
    type="text",
    prompt="""안녕하세요 텍스트프롬프트입니다 다음 {{document_type}}를 {{summury_length}} 단어이내로 요약하세요
    
    제목 : {{document_type}}
    
    내용 : {{document_content}} 형태로 요약해주세요""",
    labels=["production"],
    tags=["document", "summary", "simple", "text"],
    config={
        "model": "gpt-4.1-nano",
        "temperature":0.2,
        "max_tokens": 800
    }
)

<details>
<summary>💡 정답 보기</summary>

```python
# 간단한 문서 요약용 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="document-summarizer",
    type="text",
    prompt="""다음 {{document_type}}를 {{summary_length}} 단어 이내로 요약해주세요:

제목: {{document_title}}

내용:
{{document_content}}

{{summary_style}} 형태로 요약해주세요.""",
    labels=["production"],
    tags=["document", "summary", "simple", "text"],
    config={
        "model": "gpt-4.1-nano",
        "temperature": 0.2,
        "max_tokens": 800
    }
)
```
</details>


In [102]:
# 간단한 문서 요약용 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="document-summarizer",
    type="text",
    prompt="""다음 {{document_type}}를 {{summary_length}} 단어 이내로 요약해주세요:

제목: {{document_title}}

내용:
{{document_content}}

{{summary_style}} 형태로 요약해주세요.""",
    labels=["production"],
    tags=["document", "summary", "simple", "text"],
    config={
        "model": "gpt-4.1-nano",
        "temperature": 0.2,
        "max_tokens": 800
    }
)

In [18]:
# 챗 프롬프트 생성
# 여기에 코드를 

langfuse.create_prompt(
    name="document-summarizer-chat",
    type="chat",
    prompt= [
        {
            "role" : "system", 
            "content": "당신은 문서요약 전문가입니다. {{summary_style}} 형태로 {{summary_length}} 단어 이내로 요약해주세요 "
        },
        {"role" : "user",
        "content": """ 다음{{document_type}}를 요약해주세요 : 
        제목 : {{document_title}} 내용 :{{document_content}}"""
        }
    ],
    labels = ["production"],
    tags=["document", "summury","chat", "simple"],
    config={
        "model": "gpt-4.1-nano",
        "temperature": 0.2,
        "max_tokens": 800
    }
    
)

<details>
<summary>💡 정답 보기</summary>

```python
# 간단한 문서 요약용 챗 프롬프트 생성
langfuse.create_prompt(
    name="document-summarizer-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 문서 요약 전문가입니다. {{summary_style}} 형태로 {{summary_length}} 단어 이내로 요약해주세요."
        },
        {
            "role": "user",
            "content": """다음 {{document_type}}를 요약해주세요:

제목: {{document_title}}

내용:
{{document_content}}"""
        }
    ],
    labels=["production"],
    tags=["document", "summary", "chat", "simple"],
    config={
        "model": "gpt-4.1-nano",
        "temperature": 0.2,
        "max_tokens": 800
    }
)
```
</details>


---

## 2. 프롬프트 활용

### 2.1 기본 프롬프트 가져오기

In [29]:
# 프로덕션 버전 가져오기 프롬프트 가져오는 함수
prompt = langfuse.get_prompt("movie-critic")

# 프롬프트 정보 출력
print(f"모델: {prompt.config['model']}")
print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 호환 프롬프트 출력 랭체인문법으로 나오게 하는 문법.
print(prompt.get_langchain_prompt())

모델: gpt-4.1-nano
온도: 0.7
라벨: ['production', 'latest']
태그: ['movie', 'qa', 'text']
프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?
----------------------------------------------------------------------------------------------------
{criticLevel} 영화 평론가로서, {movie}를 어떻게 생각하시나요?


### 2.2 compile 메서드 사용

- compile 메서드로 변수 삽입

In [30]:
# compile 메서드로 프롬프트에 변수 삽입하는 방법
compiled_prompt = prompt.compile(criticLevel="전문가", movie="인셉션")
print(compiled_prompt)

전문가 영화 평론가로서, 인셉션를 어떻게 생각하시나요?


### 2.3 챗 프롬프트 가져오기 및 컴파일

In [31]:
# 챗 프롬프트 가져오기
chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

# 챗 프롬프트 정보 출력
print(f"모델: {chat_prompt.config['model']}")
print(f"온도: {chat_prompt.config['temperature']}")
print(f"라벨: {chat_prompt.labels}")
print(f"태그: {chat_prompt.tags}")
print(f"프롬프트: {chat_prompt.prompt}")
print("-" * 100)

# 랭체인 호환 프롬프트 출력
print(chat_prompt.get_langchain_prompt())

모델: gpt-4.1-nano
온도: 0.7
라벨: ['production', 'latest']
태그: ['movie', 'qa', 'chat']
프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}를 어떻게 생각하시나요?'}]
----------------------------------------------------------------------------------------------------
[('system', '당신은 {criticLevel} 영화 평론가입니다.'), ('user', '영화 {movie}를 어떻게 생각하시나요?')]


In [32]:
# 챗 프롬프트 컴파일
compiled_chat_prompt = chat_prompt.compile(criticLevel="전문가", movie="인셉션")
print(compiled_chat_prompt)

[{'role': 'system', 'content': '당신은 전문가 영화 평론가입니다.'}, {'role': 'user', 'content': '영화 인셉션를 어떻게 생각하시나요?'}]


### 2.4 메시지 플레이스홀더 활용

In [33]:
# 플레이스홀더가 있는 챗 프롬프트 가져오기
prompt_with_history = langfuse.get_prompt("movie-critic-with-history", type="chat")

# 대화 히스토리 정의
chat_history = [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "안녕하세요! 영화에 대해 이야기해볼까요?"}
]

# 변수와 플레이스홀더를 모두 컴파일
compiled_with_history = prompt_with_history.compile(
            criticLevel="전문가",
            movie="인셉션", 
            chat_history=chat_history
        )

for message in compiled_with_history:
    print(message)
    print("-" * 20)

{'role': 'system', 'content': '당신은 전문가 영화 평론가입니다2.'}
--------------------
{'role': 'user', 'content': '안녕하세요!'}
--------------------
{'role': 'assistant', 'content': '안녕하세요! 영화에 대해 이야기해볼까요?'}
--------------------
{'role': 'user', 'content': '영화 인셉션에 대해 어떻게 생각하시나요?'}
--------------------


### **[실습 2]**
"movie-critic-chat" 프롬프트를 Langfuse에서 가져와서 내용을 출력하고, compile 메서드를 사용해 변수에 적절한 값을 추가해보세요.

In [45]:
# chat 프롬프트 가져오기 및 컴파일
# 여기에 코드를 작성하세요

chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

chat_history = [{"role": "user","content": "안녕하세요!"}, {"role": "assistant","content": "안녕하세요! 영화에 대해 이야기 해볼까요?"}]

compiled_with_history = prompt_with_history.compile(
    criticLevel="일반인",
    movie = "로맨스",
    chat_history= chat_history
)

for message in compiled_with_history:
    print(message)

print(compiled_with_history)

{'role': 'system', 'content': '당신은 일반인 영화 평론가입니다2.'}
{'role': 'user', 'content': '안녕하세요!'}
{'role': 'assistant', 'content': '안녕하세요! 영화에 대해 이야기 해볼까요?'}
{'role': 'user', 'content': '영화 로맨스에 대해 어떻게 생각하시나요?'}
[{'role': 'system', 'content': '당신은 일반인 영화 평론가입니다2.'}, {'role': 'user', 'content': '안녕하세요!'}, {'role': 'assistant', 'content': '안녕하세요! 영화에 대해 이야기 해볼까요?'}, {'role': 'user', 'content': '영화 로맨스에 대해 어떻게 생각하시나요?'}]


<details>
<summary>💡 정답 보기</summary>

```python
# chat 프롬프트 가져오기 및 컴파일
chat_prompt = langfuse.get_prompt("movie-critic-chat", type="chat")

# 대화 히스토리 정의
chat_history = [
    {"role": "user", "content": "안녕하세요!"},
    {"role": "assistant", "content": "안녕하세요! 영화에 대해 이야기해볼까요?"}
]

# 변수와 플레이스홀더를 모두 컴파일
compiled_chat_prompt = chat_prompt.compile(
    criticLevel="전문가", 
    movie="인셉션"
)

for message in compiled_chat_prompt:
    print(message)
    print("-" * 20)
```
</details>


---

## 3. 프롬프트 버전 관리

### 3.1 새로운 버전 생성

In [46]:
# 새로운 버전 생성 (같은 이름 사용) 같은이름쓰면 새로운버전으로 수정됨
langfuse.create_prompt(
    name="movie-critic",  # 같은 이름 사용
    type="text",          
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}에 대한 상세한 분석을 제공해주세요. 연출, 연기, 스토리, 시각적 효과를 포함하여 평가해주세요.",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text", "detailed"],    # 태그 업데이트
    config={
        "model": "gpt-4.1-mini",  # 모델 업그레이드
        "temperature": 0.7,
        "max_tokens": 1000  # 토큰 수 증가
    }
)

### 3.2 특정 버전 가져오기

In [52]:
# 특정 버전 가져오기
prompt_v5 = langfuse.get_prompt("movie-critic", version=5)
prompt_v2 = langfuse.get_prompt("movie-critic", version=2)

# 버전별 비교
print(f"V1 프롬프트: {prompt_v5.prompt}")
print(f"V2 프롬프트: {prompt_v2.prompt}")
print(f"V1 모델: {prompt_v1.config['model']}")
print(f"V2 모델: {prompt_v2.config['model']}")

V1 프롬프트: 당신은 {{criticLevel}} 영화 평론가입니다.

영화 {{movie}}에 대한 상세한 분석을 제공해주세요. 연출, 연기, 스토리, 시각적 효과를 포함하여 평가해주세요.
V2 프롬프트: {{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?
V1 모델: gpt-4.1-nano
V2 모델: gpt-4.1-nano


### 3.3 라벨 관리

In [59]:
# 특정 라벨로 프롬프트 생성 (같은 이름을 사용하면 새로운 버전으로 생성됨)
langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "바보 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# 라벨별 프롬프트 가져오기
prompt_production = langfuse.get_prompt("movie-critic-chat", label="production")
prompt_staging = langfuse.get_prompt("movie-critic-chat", label="staging")
prompt_latest = langfuse.get_prompt("movie-critic-chat", label="latest")

In [60]:
# 라벨별 프롬프트 출력
print(f"Production 프롬프트: {prompt_production.prompt}")
print("-" * 100)
print(f"Staging 프롬프트: {prompt_staging.prompt}")
print("-" * 100)
print(f"Latest 프롬프트: {prompt_latest.prompt}")

Production 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}를 어떻게 생각하시나요?'}]
----------------------------------------------------------------------------------------------------
Staging 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요.'}, {'type': 'message', 'role': 'user', 'content': '바보 {{movie}}에 대한 평론을 작성해주세요.'}]
----------------------------------------------------------------------------------------------------
Latest 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요.'}, {'type': 'message', 'role': 'user', 'content': '바보 {{movie}}에 대한 평론을 작성해주세요.'}]


### 3.4 라벨 업데이트

In [62]:
# 기존 프롬프트 버전의 라벨 업데이트, 태그바꾸는거다
langfuse.update_prompt(
    name="movie-critic-chat",
    version=4,
    new_labels=["production", "v2-stable"]
)

Prompt_Chat(prompt=[ChatMessageWithPlaceholders_Chatmessage(role='system', content='당신은 {{criticLevel}} 영화 평론가입니다. 상세하고 전문적인 분석을 제공해주세요.', type='chatmessage'), ChatMessageWithPlaceholders_Chatmessage(role='user', content='바보 {{movie}}에 대한 평론을 작성해주세요.', type='chatmessage')], name='movie-critic-chat', version=4, config={}, labels=['production', 'v2-stable'], tags=['movie', 'qa', 'chat', 'detailed'], commit_message=None, resolution_graph=None, type='chat', createdBy='API', id='aa6fb0e5-f6bc-4261-924b-31a83ce24ed5', projectId='cmjrwsl38003bad07j6t568tc', updatedAt='2026-01-07T08:47:37.369Z', createdAt='2026-01-07T08:45:50.752Z', isActive=None)

### **[실습 3]**
"movie-critic-chat" 프롬프트를 수정하고, labels 속성은 "staging"으로 지정한 후, staging 버전을 가져와서 내용을 출력하세요.

In [66]:
# staging 라벨 생성
# 여기에 코드를 작성하세요

# staging 라벨 가져오기
# 여기에 코드를 작성하세요

langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {"role":"system",
        "content":"당신은 {{criticLevel}} 영화 평론가입니다. 일반적이고 상세하며 전문적 분석을 제공해라."}
    ,   {
        "role":"user",
        "content":"영화 {{movie}}에 대한 평론을 작성해주세요"}

    ],
    labels=["staging"],
    tags=["movie"]
)

In [68]:
prompt_staging = langfuse.get_prompt(
    name="movie-critic-chat",
    label="staging"
)


print(f"Staging 프롬프트: {prompt_staging.prompt}")

Staging 프롬프트: [{'type': 'message', 'role': 'system', 'content': '당신은 {{criticLevel}} 영화 평론가입니다. 일반적이고 상세하며 전문적 분석을 제공해라.'}, {'type': 'message', 'role': 'user', 'content': '영화 {{movie}}에 대한 평론을 작성해주세요'}]


<details>
<summary>💡 정답 보기</summary>

```python
# staging 라벨 생성
langfuse.create_prompt(
    name="movie-critic-chat",
    type="chat",
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다. 일반적이고 상세하며 전문적인 분석을 제공해주세요."
        },
        {
            "role": "user",
            "content": "영화 {{movie}}에 대한 평론을 작성해주세요."
        }
    ],
    labels=["staging"],  # staging 환경용
    tags=["movie", "qa", "chat", "detailed"]
)

# staging 라벨 가져오기
prompt_staging = langfuse.get_prompt(
    name="movie-critic-chat",
    label="staging"
)

# 라벨별 프롬프트 출력
print(f"Staging 프롬프트: {prompt_staging.prompt}")
```
</details>


---

## 4. LangChain과의 통합

### 4.1 텍스트 프롬프트와 LangChain 통합

In [ ]:
from langchain_core.prompts import PromptTemplate #텍스트프롬프트를 랭체인에서쓰고싶고 데이터는 랭퓨즈에서쓰고싶다. 
from langchain_openai import ChatOpenAI

# Langfuse 프롬프트를 LangChain과 통합
prompt = langfuse.get_prompt("movie-critic", label="production") #겟프롬프트로가져옴

langchain_prompt = PromptTemplate.from_template(
    prompt.get_langchain_prompt() #이중괄호를 랭체인스타일로바꿔준다.
)
langchain_prompt.metadata = {"langfuse_prompt": prompt}   #메타데이터 속성에다가 중괄호안에꺼쓰면 랭퓨즈가 랭퓨즈프롬프트를
#랭체인프롬프트에 메타데이터추가를하면  Langfuse 자동 링크를 위한 메타데이터

# 모델 초기화 (프롬프트 설정 사용) 
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-nano"), #딕셔너리니까 없으면  , 다음값을 사용하게  한다.
    temperature=prompt.config.get("temperature", 0.7),
    max_completion_tokens=prompt.config.get("max_tokens", 500)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백 /러너블건피그. 콜백함수로 랭퓨즈핸들러
    #핸들러를 추가해준다. 체인 실행invoke할때, 랭퓨즈 콜백함수가 이벤트들을 필요한 정보들을 뽑아서간다. 랭퓨즈에 정보 추가된다.
)

print(response.content)

영화 **《인셉션》(Inception, 2010)**은 크리스토퍼 놀란 감독의 대표작 중 하나로, 복잡한 플롯과 독창적인 시각적 연출, 깊이 있는 주제 의식으로 많은 찬사를 받은 작품입니다. 다음은 연출, 연기, 스토리, 시각적 효과 측면에서의 상세한 분석입니다.

---

### 1. 연출 (Direction)

크리스토퍼 놀란은 《인셉션》에서 꿈과 현실의 경계를 넘나드는 복잡한 내러티브를 탁월하게 구현해냈습니다. 여러 층위의 꿈 속 세계를 동시에 보여주면서도 관객이 혼란스럽지 않도록 명확한 구조를 유지하는 연출력은 놀라울 정도입니다. 특히 시간의 흐름이 각 꿈의 레벨마다 다르게 설정되어 있어, 이를 시각적·음향적으로 구분하는 세심한 연출이 돋보입니다. 또한, 액션과 심리적 긴장감 사이의 균형을 잘 맞추며, 철학적이고 심리적인 주제를 상업적 블록버스터 형식 안에 효과적으로 녹여냈습니다.

---

### 2. 연기 (Acting)

레오나르도 디카프리오가 주인공 도미닉 코브 역을 맡아, 복잡한 심리 상태와 내면의 고통을 섬세하게 표현합니다. 코브의 감정적 갈등과 죄책감, 가족에 대한 그리움이 이야기의 감성적 중심축을 이룹니다. 조셉 고든-레빗, 엘런 페이지, 톰 하디 등 조연 배우들도 각자의 역할에 깊이를 더하며, 팀 내 역학 관계와 캐릭터 개성을 잘 살려줍니다. 특히 각기 다른 꿈의 레벨에서 캐릭터들이 보여주는 다양한 감정과 태도는 관객으로 하여금 몰입도를 높입니다.

---

### 3. 스토리 (Story)

《인셉션》의 스토리는 ‘꿈속에 꿈을 심는’ 인셉션(심어주기)이라는 독특한 설정을 중심으로 펼쳐집니다. 기억과 현실, 의식과 무의식 사이의 경계가 모호해지는 가운데, 주인공의 내면적 갈등과 미션이 교차합니다. 복잡한 플롯임에도 불구하고, 놀란은 이야기의 논리성과 긴장감을 유지하며 관객이 계속해서 흥미를 느끼게 만듭니다. 다층적 꿈 세계를 통해 ‘현실이란 무엇인가’라는 철학적 질문을 던지며, 엔딩의 모호함으로 관객 각자가 해석할 여지를 남긴 점도 작품

### 4.2 챗 프롬프트와 LangChain 통합

In [72]:
from langchain_core.prompts import ChatPromptTemplate

# 챗 프롬프트 통합
chat_prompt = langfuse.get_prompt("movie-critic-chat", label="production", type="chat")

langchain_chat_prompt = ChatPromptTemplate.from_messages(
    chat_prompt.get_langchain_prompt()
)
langchain_chat_prompt.metadata = {"langfuse_prompt": chat_prompt}

# 체인 실행
chain = langchain_chat_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}
)

print(response.content)

영화 **《바보 인셉션》**은 제목에서부터 알 수 있듯, 크리스토퍼 놀란 감독의 대표작 《인셉션》을 패러디하거나 재해석한 코미디 영화로 보입니다. 다만, 실제로 이 제목을 가진 작품이 존재하는지 여부는 명확하지 않으나, 만약 존재한다면 그 작품을 가상의 시점에서 분석해보겠습니다.

### 줄거리 및 주제
《바보 인셉션》은 원작 《인셉션》의 복잡한 꿈 속 꿈의 구조를 차용하되, 이를 전혀 엉뚱하고 우스꽝스러운 상황들로 탈바꿈시킨 작품이라 할 수 있습니다. 꿈의 세계에서 벌어지는 사건들이 진지함 대신 코미디와 풍자로 채워져, 관객에게 웃음을 유발하는 동시에 원작이 던졌던 현실과 꿈, 기억과 정체성에 관한 철학적 질문을 희화화합니다.

### 연출과 연기
감독은 원작의 심오한 분위기를 전복시키는 연출을 택했습니다. 복잡한 꿈의 계층 구조를 단순하면서도 유머러스하게 표현하며, 과장된 캐릭터들의 행동과 대사는 관객에게 친근감을 줍니다. 주연 배우들의 코믹 타이밍과 표정 연기는 영화의 유머를 극대화하는 데 성공적이며, 신체 코미디와 언어유희가 적절히 조화를 이룹니다.

### 미장센과 촬영
촬영은 원작의 다층적 공간감을 희화화한 세트와 소품 디자인을 통해 시각적 재미를 제공합니다. 색채는 밝고 경쾌하며, 몽환적인 꿈의 이미지 대신 일상적이고 익살스러운 배경들이 사용되어 코미디적 효과를 강화합니다. 카메라 워킹은 원작의 긴장감 넘치는 움직임을 따라가면서도 종종 의도적으로 엉성한 구도를 취해 웃음을 유발합니다.

### 음악과 사운드 디자인
음악은 원작 한스 짐머의 웅장한 스코어 대신 경쾌하고 발랄한 멜로디를 사용하여 분위기를 가볍게 만듭니다. 사운드 효과 또한 과장된 코믹 음향을 적극 활용해 관객의 몰입감을 높입니다.

### 메시지와 의의
비록 코미디 장르에 속하지만, 《바보 인셉션》은 원작이 제기한 현실과 환상의 경계 문제를 풍자적으로 재해석합니다. 이를 통해 현대 사회의 복잡한 정보와 진실의 혼란, 그리고 우리가 믿는 ‘현실’의 불확실성을 유머러스하게 비판하는 작품으

### 4.3 플레이스홀더가 있는 프롬프트와 LangChain 통합

In [73]:
from langchain_core.prompts import MessagesPlaceholder

# 플레이스홀더가 있는 프롬프트 가져오기
prompt_with_history = langfuse.get_prompt("movie-critic-with-history", type="chat")

# LangChain 호환 프롬프트로 변환 (미해결 플레이스홀더는 MessagesPlaceholder로 변환)
langchain_prompt_with_placeholder = ChatPromptTemplate.from_messages(
    prompt_with_history.get_langchain_prompt()
)
langchain_prompt_with_placeholder.metadata = {"langfuse_prompt": prompt_with_history}

# chain 생성
chain = langchain_prompt_with_placeholder | model

# 실행 시 플레이스홀더 값 제공
chat_history = [
    {"role": "user", "content": "안녕하세요! 영화 예산에 대해서 이야기해볼까요?"},
    {"role": "assistant", "content": "안녕하세요! 영화 예산에 대해 어떻게 도와드릴까요?"}
]

response = chain.invoke({
    "criticLevel": "전문가", 
    "movie": "인셉션",
    "chat_history": chat_history
}, config={"callbacks": [langfuse_handler]})  # Langfuse 트레이싱을 위한 콜백

print(response.content)

Placeholders ['chat_history'] have not been resolved. Pass them as keyword arguments to compile().


영화 인셉션(Inception, 2010)은 크리스토퍼 놀란 감독의 대표작 중 하나로, 뛰어난 스토리텔링과 시각효과, 그리고 탄탄한 연출이 돋보이는 작품입니다. 예산 측면에서 보면, 약 1억 6천만 달러 정도가 투입된 대규모 블록버스터였는데요, 이 예산이 고스란히 영화의 높은 완성도와 몰입감으로 이어졌다고 생각합니다.

특히, 인셉션은 복잡한 꿈 속 세계를 구현하기 위해 다양한 세트와 특수효과, 그리고 실제 촬영을 병행했는데, 이는 상당한 제작비를 필요로 했습니다. 하지만 그만큼 관객들에게 신선한 시각적 경험과 깊이 있는 이야기를 제공하며, 상업적 성공과 비평적 호평을 동시에 거두었죠.

예산이 큰 영화일수록 리스크도 크지만, 인셉션은 그 투자가 결실을 맺은 좋은 사례라고 할 수 있습니다. 혹시 영화 예산과 관련해서 더 궁금한 점 있으신가요?


- 특정 라벨 가져오기

In [ ]:
# 특정 라벨 가져오기
prompt_staging = langfuse.get_prompt("movie-critic", label="latest")  # production, latest

# 프롬프트 출력
print(f"모델: {prompt_staging.config['model']}")
print(f"온도: {prompt_staging.config['temperature']}")
print(f"라벨: {prompt_staging.labels}")
print(f"프롬프트: {prompt_staging.prompt}")

In [ ]:
from langchain_core.prompts import PromptTemplate

# Langchain과 통합 
langchain_prompt = PromptTemplate.from_template(
    prompt_staging.get_langchain_prompt(),
)
langchain_prompt.metadata = {"langfuse_prompt": prompt_staging}

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-nano"),
    temperature=prompt.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

### **[실습 4]**
앞에서 정의한 텍스트 기반 프롬프트를 가져와서 LangChain과 통합하여 트레이싱을 실행하고, Langfuse UI에서 결과를 확인하세요.


In [97]:
# 여기에 코드를 추가하세요.

from langchain_core.prompts import ChatPromptTemplate # Chat용 템플릿 임포트


prompt = langfuse.get_prompt("keyword-extractor-few-shot", label ="production")

langchain_prompt = ChatPromptTemplate.from_messages(
    prompt.get_langchain_prompt()
)

langchain_prompt.metadata = {"langfuse_prompt" : prompt}


# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-nano"),
    temperature=prompt.config.get("temperature", 0.2),
    max_completion_tokens=prompt.config.get("max_tokens", 800)
)

# 체인 생성 및 실행
chain = langchain_prompt | model

# 샘플 문서 내용
sample_document = """
Langfuse는 LLM 애플리케이션을 위한 오픈소스 관찰 가능성 플랫폼입니다. 
개발자들이 LLM 애플리케이션을 디버그, 분석, 반복 개발할 수 있도록 도와줍니다.

주요 기능:
- 상세한 추적 및 관찰 가능성
- 프롬프트 관리 및 버전 제어
- 평가 및 데이터셋 관리
- 대시보드를 통한 성능 모니터링
- 다양한 프레임워크와의 통합 (LangChain, OpenAI, LangGraph 등)

Langfuse를 사용하면 프로덕션 환경에서 LLM 애플리케이션의 품질과 성능을 
지속적으로 개선할 수 있습니다.
"""

# 문서 요약 실행 (Langfuse 트레이싱 포함)
response = chain.invoke(
    input={
        "criticLevel": "기술 문서",
        "chat_history": [{"role" : "user", "content" : "나는바보야"}],
        "movie": "sample_document",
        
        
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print("요약 결과:")
print(response.content)

# Langfuse로 모든 데이터 전송 완료
langfuse.flush()



Placeholders ['chat_history'] have not been resolved. Pass them as keyword arguments to compile().


요약 결과:
안녕하세요! 영화 "sample_document"에 대해 평가를 요청하셨군요. 다만, 현재 저는 해당 영화에 대한 구체적인 정보나 내용을 가지고 있지 않아서 자세한 평가는 어려운 점 양해 부탁드립니다. 만약 영화의 줄거리, 감독, 배우, 또는 특별히 인상 깊었던 부분에 대해 알려주시면 더 구체적이고 유익한 의견을 드릴 수 있을 것 같습니다. 추가 정보를 공유해 주시면 감사하겠습니다!


<details>
<summary>💡 정답 보기</summary>

```python
# Langfuse 프롬프트를 LangChain과 통합
prompt = langfuse.get_prompt("document-summarizer", label="production")

langchain_prompt = PromptTemplate.from_template(
    prompt.get_langchain_prompt()
)

# Langfuse 자동 링크를 위한 메타데이터 설정
langchain_prompt.metadata = {"langfuse_prompt": prompt}

# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-nano"),
    temperature=prompt.config.get("temperature", 0.2),
    max_completion_tokens=prompt.config.get("max_tokens", 800)
)

# 체인 생성 및 실행
chain = langchain_prompt | model

# 샘플 문서 내용
sample_document = """
Langfuse는 LLM 애플리케이션을 위한 오픈소스 관찰 가능성 플랫폼입니다. 
개발자들이 LLM 애플리케이션을 디버그, 분석, 반복 개발할 수 있도록 도와줍니다.

주요 기능:
- 상세한 추적 및 관찰 가능성
- 프롬프트 관리 및 버전 제어
- 평가 및 데이터셋 관리
- 대시보드를 통한 성능 모니터링
- 다양한 프레임워크와의 통합 (LangChain, OpenAI, LangGraph 등)

Langfuse를 사용하면 프로덕션 환경에서 LLM 애플리케이션의 품질과 성능을 
지속적으로 개선할 수 있습니다.
"""

# 문서 요약 실행 (Langfuse 트레이싱 포함)
response = chain.invoke(
    input={
        "document_type": "기술 문서",
        "summary_length": "100",
        "document_title": "Langfuse 소개",
        "document_content": sample_document,
        "summary_style": "불릿포인트"
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print("요약 결과:")
print(response.content)

# Langfuse로 모든 데이터 전송 완료
langfuse.flush()
```
</details>


In [107]:

from langchain_core.prompts import ChatPromptTemplate # Chat용 템플릿 임포트

# Langfuse 프롬프트를 LangChain과 통합
prompt = langfuse.get_prompt("document-summarizer", label="production")

langchain_prompt = ChatPromptTemplate.from_messages(
    prompt.get_langchain_prompt(),
    template_format="mustache"
)

# Langfuse 자동 링크를 위한 메타데이터 설정
langchain_prompt.metadata = {"langfuse_prompt": prompt}

# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-nano"),
    temperature=prompt.config.get("temperature", 0.2),
    max_completion_tokens=prompt.config.get("max_tokens", 800)
)

# 체인 생성 및 실행
chain = langchain_prompt | model

# 샘플 문서 내용
sample_document = """
Langfuse는 LLM 애플리케이션을 위한 오픈소스 관찰 가능성 플랫폼입니다. 
개발자들이 LLM 애플리케이션을 디버그, 분석, 반복 개발할 수 있도록 도와줍니다.

주요 기능:
- 상세한 추적 및 관찰 가능성
- 프롬프트 관리 및 버전 제어
- 평가 및 데이터셋 관리
- 대시보드를 통한 성능 모니터링
- 다양한 프레임워크와의 통합 (LangChain, OpenAI, LangGraph 등)

Langfuse를 사용하면 프로덕션 환경에서 LLM 애플리케이션의 품질과 성능을 
지속적으로 개선할 수 있습니다.
"""

# 문서 요약 실행 (Langfuse 트레이싱 포함)
response = chain.invoke(
    input={
        "document_type": "기술 문서",
        "summary_length": "100",
        "document_title": "Langfuse 소개",
        "document_content": sample_document,
        "summary_style": "불릿포인트"
    },
    config={"callbacks": [langfuse_handler]}  # Langfuse 트레이싱을 위한 콜백
)

print("요약 결과:")
print(response.content)

# Langfuse로 모든 데이터 전송 완료
langfuse.flush()



요약 결과:
안녕하세요! 무엇을 도와드릴까요?


### **[실습 5]**
앞에서 정의한 chat 기반 프롬프트를 가져와서 LangChain과 통합하여 트레이싱을 실행하고, Langfuse UI에서 결과를 확인하세요.

In [ ]:
# 여기에 코드를 추가하세요.

<details>
<summary>💡 정답 보기</summary>

```python
# Langfuse 챗 프롬프트를 LangChain과 통합
chat_prompt = langfuse.get_prompt("document-summarizer-chat", type="chat")

# ChatPromptTemplate로 변환 (챗 메시지 형태)
langchain_chat_prompt = ChatPromptTemplate.from_messages(
    chat_prompt.get_langchain_prompt()
)

# Langfuse 자동 링크를 위한 메타데이터 설정
langchain_chat_prompt.metadata = {"langfuse_prompt": chat_prompt}

# 모델 초기화 (프롬프트 설정 사용)
model = ChatOpenAI(
    model=chat_prompt.config.get("model", "gpt-4.1-nano"),
    temperature=chat_prompt.config.get("temperature", 0.2),
    max_completion_tokens=chat_prompt.config.get("max_tokens", 800)
)

# 체인 생성
chain = langchain_chat_prompt | model

# 샘플 문서 내용들
sample_documents = {
    "technical": """
Langfuse는 LLM 애플리케이션을 위한 오픈소스 관찰 가능성 플랫폼입니다. 
개발자들이 LLM 애플리케이션을 디버그, 분석, 반복 개발할 수 있도록 도와줍니다.

주요 기능:
- 상세한 추적 및 관찰 가능성: 모든 LLM 호출과 체인 실행을 추적
- 프롬프트 관리 및 버전 제어: 프롬프트의 버전 관리와 A/B 테스트
- 평가 및 데이터셋 관리: 모델 성능 평가와 벤치마크 데이터셋
- 대시보드를 통한 성능 모니터링: 실시간 성능 지표와 사용량 분석
- 다양한 프레임워크와의 통합: LangChain, OpenAI, LangGraph 등과 원활한 통합

Langfuse를 사용하면 프로덕션 환경에서 LLM 애플리케이션의 품질과 성능을 
지속적으로 개선할 수 있으며, 팀 간의 협업도 효율적으로 진행할 수 있습니다.
""",
    "meeting": """
2024년 3월 주간 개발팀 회의록

참석자: 김개발, 이기획, 박디자인, 최테스트
일시: 2024년 3월 15일 오후 2시

주요 안건:
1. 프로젝트 진행 상황 점검
   - 백엔드 API 개발 90% 완료
   - 프론트엔드 UI 구현 70% 완료
   - 테스트 케이스 작성 60% 완료

2. 이슈 및 해결 방안
   - 데이터베이스 성능 최적화 필요
   - 사용자 인증 모듈 버그 수정 완료
   - 모바일 반응형 디자인 개선 진행 중

3. 다음 주 계획
   - 통합 테스트 시작
   - 사용자 피드백 수집
   - 배포 준비 작업

회의 종료 시간: 오후 3시 30분
"""
}

# 문서 요약 실행 

response = chain.invoke(
    input={
        "summary_style": "불릿포인트",
        "summary_length": "150",
        "document_type": "기술 문서",
        "document_title": "Langfuse 플랫폼 소개",
        "document_content": sample_documents["technical"]
    },
    config={"callbacks": [langfuse_handler]}
)

print("요약 결과:")
print(response.content)

# 모든 데이터를 Langfuse로 전송
langfuse.flush()

```
</details>
